In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table
import psycopg2
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

In [2]:
csv = 'gun-violence-data_01-2013_03-2018.csv'
guns_db = pd.read_csv(csv)
guns_db

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239672,1083142,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,http://www.gunviolencearchive.org/incident/108...,http://www.klfy.com/news/local/rayne-woman-cha...,False,...,0::25,0::Adult 18+,0::Female,0::Jhkeya Tezeno,NaN,"0::Unharmed, Arrested",0::Subject-Suspect,http://www.klfy.com/news/local/rayne-woman-cha...,NaN,NaN
239673,1083139,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,http://www.gunviolencearchive.org/incident/108...,http://www.ksla.com/story/37854648/man-wanted-...,False,...,1::21,0::Adult 18+||1::Adult 18+,0::Male||1::Male,0::Jamal Haskett||1::Jaquarious Tyjuan Ardison,NaN,"0::Killed||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,http://www.ksla.com/story/37854648/man-wanted-...,23.0,31.0
239674,1083151,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,http://www.gunviolencearchive.org/incident/108...,http://www.nola.com/crime/index.ssf/2018/04/sh...,False,...,0::21,0::Adult 18+,0::Male,NaN,NaN,0::Injured,0::Victim,http://www.nola.com/crime/index.ssf/2018/04/sh...,85.0,7.0
239675,1082514,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,http://www.gunviolencearchive.org/incident/108...,https://www.ch

In [18]:
guns_db = guns_db[['date', 'state', 'n_killed', 'n_injured']]
guns_db

,date,state,n_killed,n_injured
105711,2016-01-01,Massachusetts,0,1
105712,2016-01-01,Connecticut,0,0
105713,2016-01-01,Missouri,0,1
105714,2016-01-01,Missouri,0,1
105715,2016-01-01,Wisconsin,2,0
...,...,...,...,...
225870,2017-12-31,South Carolina,0,0
225871,2017-12-31,Florida,0,0
225872,2017-12-31,Florida,0,1
225873,2017-12-31,Illinois,0,1


In [4]:
guns_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239677 entries, 0 to 239676
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   date       239677 non-null  object
 1   state      239677 non-null  object
 2   n_killed   239677 non-null  int64 
 3   n_injured  239677 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 7.3+ MB


In [19]:
guns_db = guns_db[(guns_db['date']>'2015-12-31') & (guns_db['date']<'2018-01-01')]

In [20]:
guns_db[['year','month', 'day']] = guns_db.date.str.split("-",expand=True)

In [ ]:
guns_db

In [21]:
new_db = guns_db.groupby(['state', 'year']).agg('sum').sort_values(["year"], ascending=True)
new_db

,,n_killed,n_injured
state,year,,
Alabama,2016,488,761
Maine,2016,18,37
Maryland,2016,429,748
Massachusetts,2016,112,468
Michigan,2016,385,700
...,...,...,...
North Carolina,2017,556,1153
Kansas,2017,152,264
North Dakota,2017,15,33


In [22]:
new_db.reset_index('year', inplace=True)
new_db

,year,n_killed,n_injured
state,,,
Alabama,2016,488,761
Maine,2016,18,37
Maryland,2016,429,748
Massachusetts,2016,112,468
Michigan,2016,385,700
...,...,...,...
North Carolina,2017,556,1153
Kansas,2017,152,264
North Dakota,2017,15,33


In [23]:
guns_2016 = new_db.loc[new_db['year']=='2016']


,year,n_killed,n_injured
state,,,
Alabama,2016,488,761
Maine,2016,18,37
Maryland,2016,429,748
Massachusetts,2016,112,468
Michigan,2016,385,700
Minnesota,2016,108,217
Mississippi,2016,290,497
Wyoming,2016,18,12
Montana,2016,42,22


In [24]:
guns_2017 = new_db.loc[new_db['year']=='2017']

In [25]:
guns_2016 = guns_2016.drop(columns=['year'])
guns_2016

,n_killed,n_injured
state,,
Alabama,488,761
Maine,18,37
Maryland,429,748
Massachusetts,112,468
Michigan,385,700
Minnesota,108,217
Mississippi,290,497
Wyoming,18,12
Montana,42,22


In [26]:
guns_2017 = guns_2017.drop(columns=['year'])
guns_2017.head()

,n_killed,n_injured
state,,
Vermont,9,11
Connecticut,89,293
Oregon,99,170
Pennsylvania,635,1188
Colorado,199,300


In [27]:
guns_2016.to_csv('2016_gun_violence.csv')

In [28]:
guns_2017.to_csv('2017_gun_violence.csv')